# 03b - Vertex AI + BQML - Online Predictions with BQML Models

Models built with BigQuery ML (BQML), like the one in (03a), can also be exported for use and deployment outside of BigQuery.  A Vertex AI Endpoint can be used for online predictions with an exported model.  This demonstration shows the process of exporting and deploying a BQML model with Vertex AI.

### Video Walkthrough of this notebook:
Includes conversational walkthrough and more explanatory information than the notebook:

<p><center><a href="https://youtu.be/7y_t_bW0LHQ" target="_blank" rel="noopener noreferrer"><img src="architectures/thumbnails/playbutton/03b.png" width="50%"></a></center></p>

### Prerequisites:
-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

### Overview:
-  Export the BigQuery ML model built in (03a) to a GCS bucket URI
   -  EXPORT MODEL …
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict

### Resources:
-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/03b_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/03b_console.png">

---
## Setup

inputs:

In [16]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [17]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '03b'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [18]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [19]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [20]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}
DIR = f"temp/{NOTEBOOK}"

environment:

In [21]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [24]:
export = bq.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")

In [25]:
export.result()

---
## Serving

### Upload The Model
https://googleapis.dev/python/aiplatform/latest/aiplatform.html?highlight=aiplatform%20model%20upload#google.cloud.aiplatform.Model.upload

In [26]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/3800910542151876608/operations/8981632929335607296
Model created. Resource name: projects/1026793852137/locations/us-central1/models/3800910542151876608@1
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/3800910542151876608@1')


In [27]:
model.display_name

'03b_fraud_20220707104636'

---
**UPDATE**

As of [April 1, 2022], the BigQuery model can be directly exported to Vertex AI at training (preview feature).  This has the added benefit of also staying synced - if you remove the model in BigQuery it is also removed in Vertex AI.

[More Details](https://cloud.google.com/bigquery-ml/docs/managing-models-vertex)

```PYTHON
query = f"""
CREATE OR REPLACE MODEL `{DATANAME}.{DATANAME}_lr2`
OPTIONS (
    model_type = 'LOGISTIC_REG',
    auto_class_weights = TRUE,
    input_label_cols = ['{VAR_TARGET}'],
    HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT',
    HPARAM_TUNING_OBJECTIVES = ['ROC_AUC'],
    l1_reg=hparam_range(0, 10),
    MAX_PARALLEL_TRIALS = 2,
    NUM_TRIALS = 20,
    data_split_col = 'custom_splits',
    data_split_method = 'CUSTOM',
    model_registry="vertex_ai",
    vertex_ai_model_id='{DATANAME}_lr',
    vertex_ai_model_version_aliases=['logistic_reg', 'hyperparameter=l1_reg']
    ) AS
SELECT * EXCEPT({','.join(VAR_OMIT.split())}, splits),
    CASE
        WHEN splits = 'TRAIN' THEN FALSE
        ELSE TRUE
    END AS custom_splits
FROM `{DATANAME}.{DATANAME}_prepped`
WHERE splits != 'TEST'
"""
job = bq.query(query = query)
job.result()
(job.ended-job.started).total_seconds()
```

---

### Create An Endpoint

In [32]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

Creating Endpoint
Create Endpoint backing LRO: projects/1026793852137/locations/us-central1/endpoints/6922833071733473280/operations/3108939015244480512
Endpoint created. Resource name: projects/1026793852137/locations/us-central1/endpoints/6922833071733473280
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/6922833071733473280')


In [33]:
endpoint.display_name

'03b_fraud_20220707104636'

### Deploy Model To Endpoint

In [34]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

Deploying Model projects/1026793852137/locations/us-central1/models/3800910542151876608 to Endpoint : projects/1026793852137/locations/us-central1/endpoints/6922833071733473280
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/6922833071733473280/operations/3649370970528940032


/opt/conda/lib/python3.7/site-packages/proto/marshal/rules/enums.py:40: UserWarning: Unrecognized DeploymentResourcesType enum value: 3
  value=value,


Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/6922833071733473280


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [36]:
pred = bq.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [37]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e9d16028-4b41-4753-87ee-041d33642ae9,TEST
1,35483,1.286640,0.072917,0.212182,-0.269732,-0.283961,-0.663306,-0.016385,-0.120297,-0.135962,...,0.052674,0.076792,0.209208,0.847617,-0.086559,-0.008262,0.0,0,8b319d3a-2b2d-445b-a9a2-0da3d664ec2a,TEST
2,163935,1.961967,-0.247295,-1.751841,-0.268689,0.956431,0.707211,0.020675,0.189433,0.455055,...,0.186420,-1.621368,-0.131098,0.034276,-0.004909,-0.090859,0.0,0,788afb87-60aa-4482-8b48-c924bec634aa,TEST
3,30707,-0.964364,0.176372,2.464128,2.672539,0.145676,-0.152913,-0.591983,0.305066,-0.148034,...,-0.024200,0.365226,-0.745369,-0.060544,0.095692,0.217639,0.0,0,473d0936-1974-4ae8-ab70-230e7599bd3f,TEST


In [38]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET,'splits'])]].to_dict(orient='records')[0]
#newob

In [39]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [40]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'Class_probs': [0.1831418921518111, 0.8168581078481889], 'Class_values': ['1', '0'], 'predicted_Class': ['0']}], deployed_model_id='2334909698605580288', model_version_id='', model_resource_name='projects/1026793852137/locations/us-central1/models/3800910542151876608', explanations=None)

In [41]:
prediction.predictions[0]#['classes'][np.argmax(prediction.predictions[0]['scores'])]

{'Class_probs': [0.1831418921518111, 0.8168581078481889],
 'Class_values': ['1', '0'],
 'predicted_Class': ['0']}

In [42]:
prediction.predictions[0][f'{VAR_TARGET}_values'][np.argmax(prediction.predictions[0][f'{VAR_TARGET}_probs'])]

'0'

### Get Predictions: REST

In [43]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [44]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "Class_values": [
        "1",
        "0"
      ],
      "predicted_Class": [
        "0"
      ],
      "Class_probs": [
        0.18314189215181109,
        0.81685810784818891
      ]
    }
  ],
  "deployedModelId": "2334909698605580288",
  "model": "projects/1026793852137/locations/us-central1/models/3800910542151876608",
  "modelDisplayName": "03b_fraud_20220707104636"
}


### Get Predictions: gcloud (CLI)

In [45]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'Class_probs': [0.1831418921518111, 0.8168581078481889], 'Class_values': ['1', '0'], 'predicted_Class': ['0']}]


---
## Remove Resources
see notebook "99 - Cleanup"